In [1]:
import time
from loguru import logger
import pandas as pd
from tqdm import tqdm
import difflib
from tools.utils import get_type_from_label,get_prompt,post_process,process_label
from tools.inference import load_model_and_tokenizer,glm3_inference,glm4_inference
logger.remove()
logger.add(f"{time.strftime('%y%m%d-%H%M%S.log')}")

1

In [2]:
DATA_SELECT = 'dev'
GLM3_MODEL_PATH = '/root/autodl-tmp/checkpoint-5000'
GLM4_MODEL_PATH = '/root/output/checkpoint-5000'
start_idx = 0

In [3]:
df = pd.read_excel(f'data/data-0520/{DATA_SELECT}.xlsx')
stock_names = pd.read_excel('data/data-0520/标准名.xlsx',sheet_name='股票标准名')['标准股票名称'].to_list()
fund_names = pd.read_excel('data/data-0520/标准名.xlsx',sheet_name='基金标准名')['标准基金名称'].to_list()
all_names = stock_names + fund_names

glm3_finetune,tokenizer = load_model_and_tokenizer(GLM3_MODEL_PATH)
glm4_lora,tokenizer2 = load_model_and_tokenizer(GLM4_MODEL_PATH)

df['glm3_output'] = ''
df['glm3_output_'] = ''
df['type_'] = 0
df['prompt'] = ''
df['prompt2'] = ''
df['glm4_output'] = ''
df['glm4_output2'] = ''
df['final_output'] = ''
df['final_output2'] = ''

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
for idx,row in tqdm(df.iterrows(),total=df.shape[0]):

    if idx not in (8,10,19,23,
                  12,13,14,15,16,21):
        continue
    
    if idx<start_idx:
        continue
        
    query = row['query']
    label = row['label']
    products_str = '、'.join(difflib.get_close_matches(query,all_names,n=50,cutoff=0.0001))
    glm3_input = f'你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：{query} \n query中提到的产品标准名可能是：{products_str}'
    
    glm3_output = glm3_inference(glm3_input,glm3_finetune,tokenizer)
    df.at[idx,'glm3_output'] = glm3_output    
    type_       = get_type_from_label(glm3_output)
    glm3_output_ = process_label(glm3_output) # 个别原始小错处理
    
    glm4_prompts = get_prompt(query,type_,glm3_output_,stock_names,fund_names)
    
    message = {'role':'user','content':glm4_prompts['normal_prompt']}
    glm4_output = glm4_inference(message,glm4_lora,tokenizer2)
    final_output = post_process(glm4_output)
    
    message = {'role':'user','content':glm4_prompts['correct_prompt']}
    glm4_output2 = glm4_inference(message,glm4_lora,tokenizer2)
    final_output2 = post_process(glm4_output2)

    # if idx % 100 == 0:
    logger.info( f' {r"="*30} 第 {idx:04d} 条数据  {r"="*30} ')
    logger.info(f'Query: {query}')
    logger.info(f'GLM3 Answer: {glm3_output}')
    logger.info(f'GLM4 Answer: {glm4_output}')
    logger.info(f'GLM4 Answer2：{glm4_output2}')
    logger.info(f'Label: {label}')
        # df.to_parquet('output.parq')

    df.iloc[idx,:] = [query,label,glm3_output,glm3_output_,type_,glm4_prompts['normal_prompt'],glm4_prompts['correct_prompt'],
                      glm4_output,glm4_output2,final_output,final_output2]
    df.iloc[start_idx:idx+1,:].to_excel(f'{DATA_SELECT}_output.xlsx')
    
    # 输出到 submit.jsonl
    if 'test' in DATA_SELECT:
        with open('submit.jsonl','a') as f:
            f.write(final_output+'\n')
        with open('submit2.jsonl','a') as f:
            f.write(final_output2+'\n')

100%|██████████| 1092/1092 [09:02<00:00,  2.01it/s]


In [6]:
df.to_excel('train.xlsx')

In [7]:
df.iloc[0:]

,query,label,glm3_output,glm3_output_,type_,prompt,prompt2,glm4_output,glm4_output2,final_output,final_output2
0,我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
1,我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
2,我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
3,南方新能源C的基金经理是谁，他管理的基金总规模有多大？,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
4,汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
1087,我想知道我三个月前购买的欧普股票，如果今天卖出，净赚收入是多少，我当时购买的股票价格是每股1...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
1088,华安新动力的成立年限是多久，同时我想知道它的蚂蚁金选标识和最近三年的年化收益率同类排名。,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
1089,我想用10000块买安信动态A，我需要支付多少申购费用,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,
1090,今天金现代的最高价是多少？,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...",,,0,,,,,,


---